<a href="https://colab.research.google.com/github/adilabduakhanov/Developing_Neuro-Employees_on_GPT/blob/main/%D0%9D%D0%B5%D0%B9%D1%80%D0%BE_%D0%BA%D0%BE%D0%BD%D1%82%D1%80%D0%BE%D0%BB%D1%8C_%D0%BA%D0%B0%D1%87%D0%B5%D1%81%D1%82%D0%B2%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai==1.57.0 langchain==0.3.9 langchain_core==0.3.21 \
langchain_community==0.3.9 langchain-openai==0.2.11 \
faiss-cpu==1.9.0.post1 tiktoken==0.8.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.7 requires lang

In [ ]:
import os
import re
import json
import textwrap
import tiktoken

from google.colab import userdata, output
from openai import OpenAI

from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings


In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
client = OpenAI()


In [ ]:
folder_url_yadisk = "https://disk.yandex.ru/d/IqOQeMnvd6YSKg"
full_url = f'https://getfile.dokpub.com/yandex/get/{folder_url_yadisk}'
temp_dir = 'temp'

def download_from_url(full_url, temp_dir):
    if not os.path.exists(temp_dir):
        os.mkdir(temp_dir)

    with open("temp/tmp.txt", "w") as f:
        f.write(full_url)

    !wget -O temp.zip -i temp/tmp.txt
    !unzip -o temp.zip -d /content/
    output.clear()
    print("Файлы успешно загружены!")

download_from_url(full_url, temp_dir)


Файлы успешно загружены!


In [ ]:
def load_document_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

text_dialog = load_document_text('/content/Audio Record/merged_dialogue.txt')
text_manager = load_document_text('/content/Audio Record/manager_recogn.txt')
text_client = load_document_text('/content/Audio Record/client_recogn.txt')


In [ ]:
def create_db_index(text, chunk_size=2000):
    splitter = CharacterTextSplitter(chunk_size=chunk_size,
                                     chunk_overlap=0,
                                     separator="\n")
    chunks = splitter.split_text(text)
    return FAISS.from_texts(chunks, OpenAIEmbeddings())

dialog_db = create_db_index(text_dialog)
manager_db = create_db_index(text_manager)
client_db = create_db_index(text_client)


In [ ]:
def analyze_text(system_prompt, instructions, topic_phrase, db, k=5):
    docs = db.similarity_search(topic_phrase, k=k)
    context = "\n".join([doc.page_content for doc in docs])

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"{instructions}\n\nТексты:\n{context}"}
    ]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.2
    )

    return response.choices[0].message.content


In [ ]:
system_prompt = """
Ты эксперт по контролю качества отдела продаж.
"""

instructions = """
Проанализируй:
1. Насколько структурировано менеджер вел диалог
2. Выявлял ли потребности
3. Использовал ли аргументацию
4. Работал ли с возражениями
5. Насколько уверенно продавал

Дай оценку от 0% до 100% и объясни почему.
"""

sales_analysis = analyze_text(
    system_prompt,
    instructions,
    "продажа аргументы потребности возражения",
    dialog_db
)

print(sales_analysis)


### Оценка менеджера по продажам

1. **Структурированность диалога: 75%**
   - Менеджер ведет диалог достаточно структурировано, задавая последовательные вопросы и уточняя информацию. Однако, в некоторых местах диалог может показаться немного разрозненным, особенно когда менеджер переключается между темами. Например, в конце диалога менеджер начинает говорить о тарифах, не завершив обсуждение потребностей клиента.

2. **Выявление потребностей: 85%**
   - Менеджер активно задает вопросы, чтобы понять потребности клиента, такие как желание минимизировать человеческий фактор в работе и важность дохода. Он также уточняет, что клиент ищет в новой профессии, что свидетельствует о хорошем уровне выявления потребностей.

3. **Использование аргументации: 70%**
   - Менеджер использует некоторые аргументы, чтобы подчеркнуть преимущества работы в новой сфере, такие как возможность удаленной работы и высокий доход. Однако, аргументация могла бы быть более убедительной и конкретной, особенно в отно

In [ ]:
instructions_tariffs = """
Проанализируй:
1. Были ли названы тарифы
2. Были ли различия между тарифами
3. Понятно ли объяснена ценность тарифов
4. Насколько убедительно они были представлены

Дай оценку от 0% до 100%.
"""

tariff_analysis = analyze_text(
    system_prompt,
    instructions_tariffs,
    "тариф стоимость пакет программа",
    manager_db
)

print(tariff_analysis)


На основе предоставленных текстов, вот анализ по заданным критериям:

1. **Были ли названы тарифы**: Да, тарифы были названы: Light, PRO и PRO MAX. 
   - **Оценка**: 100%

2. **Были ли различия между тарифами**: Да, различия между тарифами были четко обозначены. Например, тариф PRO предлагает дополнительные занятия и стажировки, а PRO MAX включает индивидуальные Zoom-консультации. 
   - **Оценка**: 100%

3. **Понятно ли объяснена ценность тарифов**: В целом, ценность тарифов была объяснена, но местами информация могла быть более структурированной. Например, упоминание о доходах и возможностях трудоустройства было полезным, но некоторые детали могли быть более ясными для потенциального клиента.
   - **Оценка**: 80%

4. **Насколько убедительно они были представлены**: Менеджер использовал примеры и конкретные цифры, что делает представление тарифов более убедительным. Однако, в некоторых местах можно было бы улучшить аргументацию, чтобы сделать предложение более привлекательным.
   - **О

In [ ]:
instructions_payment = """
Проанализируй:
1. Были ли предложены варианты оплаты
2. Объяснил ли менеджер рассрочку / кредит
3. Насколько понятно описал условия оплаты
4. Насколько это способствовало продаже

Дай оценку от 0% до 100%.
"""

payment_analysis = analyze_text(
    system_prompt,
    instructions_payment,
    "оплата рассрочка кредит цена стоимость",
    manager_db
)

print(payment_analysis)


На основе предоставленного текста, я проведу анализ по указанным критериям:

1. **Были ли предложены варианты оплаты**: 
   - Менеджер упоминает о возможности единовременной оплаты и банковской рассрочки, что говорит о наличии нескольких вариантов оплаты. 
   - **Оценка: 90%** (варианты предложены, но не все детали обсуждены).

2. **Объяснил ли менеджер рассрочку / кредит**: 
   - Менеджер упоминает о рассрочке и предлагает подать заявку на неё, однако не предоставляет подробной информации о процессе получения рассрочки или условиях. 
   - **Оценка: 70%** (рассрочка упомянута, но объяснение недостаточно подробное).

3. **Насколько понятно описал условия оплаты**: 
   - Условия оплаты упоминаются, но не все аспекты (например, проценты по рассрочке, сроки) объяснены. Менеджер делает акцент на скидке, но не на всех условиях. 
   - **Оценка: 60%** (условия описаны, но недостаточно ясно и полно).

4. **Насколько это способствовало продаже**: 
   - Менеджер активно обсуждает преимущества обу

In [ ]:
final_system = """
Ты руководитель отдела контроля качества продаж.
Составь профессиональный итоговый отчет.
"""

final_instructions = f"""
На основе анализа:

1. Продажа:
{sales_analysis}

2. Тарифы:
{tariff_analysis}

3. Оплата:
{payment_analysis}

Составь структурированный отчет:

1. Общая оценка менеджера
2. Сильные стороны
3. Зоны роста
4. Общий процент качества продажи
5. Рекомендации
"""

final_report = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": final_system},
        {"role": "user", "content": final_instructions}
    ],
    temperature=0.2
)

print("\n===== ИТОГОВЫЙ ОТЧЕТ =====\n")
print(textwrap.fill(final_report.choices[0].message.content, 120))



===== ИТОГОВЫЙ ОТЧЕТ =====

# Итоговый отчет по контролю качества продаж  ## 1. Общая оценка менеджера Общая оценка работы менеджера по продажам
составляет **77%**. Менеджер демонстрирует хорошие навыки в структурировании диалога и выявлении потребностей клиента,
однако есть области, требующие улучшения.  ## 2. Сильные стороны - **Структурированность диалога (75%)**: Менеджер ведет
диалог последовательно, задавая уточняющие вопросы. - **Выявление потребностей (85%)**: Активное использование вопросов
для понимания потребностей клиента. - **Работа с возражениями (80%)**: Эффективное управление сомнениями клиента, что
способствует снижению уровня тревожности. - **Уверенность в продаже (75%)**: Демонстрация уверенности в предложениях и
активное использование рассрочки.  ## 3. Зоны роста - **Использование аргументации (70%)**: Аргументация могла бы быть
более убедительной и конкретной, особенно в отношении результатов обучения. - **Завершение сделки**: Менеджер
недостаточно настойчив в зав